# Conciertos

---
## Extracción de datos

Vamos a hacer un scrapping de SongKick.

Necesitamos:
- Convertir los artistas del top a su ID de songkick
- Recorrer los resultados

**0 -> LIBRERÍAS**

In [42]:
# INSTALACION

# !pip install beautifulsoup4
# !pip install requests
# !pip install splinter
# !pip install lxml
# !pip install pandas
# !pip install numpy
# !pip install requests-ip-rotator

# IMPORTACION (EJECUTAR SIEMPRE)

from bs4 import BeautifulSoup as bs
import requests
# from requests_ip_rotator import ApiGateway, EXTRA_REGIONS
from splinter import Browser

import pandas as pd
import numpy as np
import json
import timeit
from time import sleep
import time
import datetime
import os
from dateutil import parser


In [83]:
URL_BASE = 'https://www.songkick.com/artists/'
GIGOGRAPHY = '/gigography'

DICT_ARTIST_WORLD = {
    # SPOTIFY - SONGKICK
    'Adele': '537914-adele',
    'Bad Bunny': '9059639-bad-bunny',
    'Becky G': '7061074-becky-g',
    'Bizarrap': '9233254-bizarrap',
    'Camila Cabello': '8504398-camila-cabello',
    'Coldplay': '197928-coldplay',
    'Cris Mj': '10173899-cris-mj',
    'Doja Cat': '8030683-doja-cat',
    'Ed Sheeran': '2083334-ed-sheeran',
    'Elley Duhé': '1236952-elley-duhe',
    'Elton John': '371163-elton-john',
    'Farruko': '3780971-farruko',
    'GAYLE': '482649-gayle',
    'Glass Animals': '1406962-glass-animals',
    'Harry Styles': '5820634-harry-styles',
    'Imagine Dragons': '2506696-imagine-dragons',
    'Jack Harlow': '8899609-jack-harlow',
    'Jaymes Young': '7001609-jaymes-young',
    'Joji': '9214309-joji',
    'Justin Bieber': '2596951-justin-bieber',
    'KAROL G': '9254044-karol-g',
    'Kate Bush': '429022-kate-bush',
    'Lil Nas X': '10001194-lil-nas-x',
    'Lizzo': '5031783-lizzo',
    'Lost Frequencies': '8423943-lost-frequencies',
    'Manuel Turizo': '9465704-manuel-turizo',
    'Olivia Rodrigo': '10112623-olivia-rodrigo',
    'OneRepublic': '568431-onerepublic',
    'Rauw Alejandro': '9920644-rauw-alejandro',
    'Ruth B.': '8675084-ruth-b',
    'Stephen Sanchez': '10218072-stephen-sanchez',
    'Tainy': '10010114-tainy',
    'The Kid LAROI': '9784044-kid-laroi',
    'The Neighbourhood': '636848-neighbourhood',
    'The Walters': '5878244-walters',
    'The Weeknd': '4363463-weeknd',
    'Tiësto': '152971-tiesto',
    'Tom Odell': '4582588-tom-odell',
}

DICT_ARTIST_PRUEBA = {
    # SPOTIFY - SONGKICK
    'Adele': '537914-adele',
    'Bad Bunny': '9059639-bad-bunny',
    'Becky G': '7061074-becky-g',
}

DICT_ARTIST_SPAIN = {
    'Aitana': '9500809-aitana',
    'Alejo': '125806-alejo',
    'Ana Mena': '9106564-ana-mena',
    'Bad Bunny': '9059639-bad-bunny',
    'Bad Gyal': '8901654-bad-gyal',
    'Becky G': '7061074-becky-g',
    'Beny Jr': '10178566-beny-jr',
    'Bizarrap': '9233254-bizarrap',
    'Blessd': '10159545-blessd',
    'C. Tangana': '8692633-c-tangana',
    'Camila Cabello': '8504398-camila-cabello',
    'Cano': '249665-cano',
    'Chanel': '607616-chanel',
    'Chema Rivas': '10133758-chema-rivas',
    'Chimbala': '5588963-chimbala',
    'Chris Jedi': '10231843-chris-jedi',
    'Cris Mj': '10173899-cris-mj',
    'Daddy Yankee': '390360-daddy-yankee',
    'Danny Ocean': '9432294-danny-ocean',
    'Daviles de Novelda': '10100622-daviles-de-novelda',
    'Duki': '9235114-duki',
    'Elton John': '371163-elton-john',
    'Farruko': '3780971-farruko',
    'Feid': '2107217-feid',
    'Fuego': '267269-fuego',
    'Harry Styles': '5820634-harry-styles',
    'Justin Quiles': '8838644-justin-quiles',
    'KAROL G': '9254044-karol-g',
    'La La Love You': '975950-la-la-love-you',
    'La Pantera': '10195304-la-pantera',
    'Luar La L': '10170197-luar-la-l',
    'Manuel Turizo': '9465704-manuel-turizo',
    'Marc Seguí': '10105446-marc-segui',
    'Morad': '1252016-morad',
    'Myke Towers': '9734829-myke-towers',
    'Omar Montes': '9661509-omar-montes',
    'Oscu': '10205892-oscu',
    'Paulo Londra': '9307999-paulo-londra',
    'Polimá Westcoast': '9578634-polima-westcoast',
    'Quevedo': '10191780-quevedo',
    'Rauw Alejandro': '9920644-rauw-alejandro',
    'Rels B': '9316164-rels-b',
    'Rigoberta Bandini': '10141428-rigoberta-bandini',
    'ROSALÍA': '786955-rosalia',
    'Sebastian Yatra': '8981304-sebastian-yatra',
    'Shakira': '8981304-sebastian-yatra',
    'Tiago PZK': '10054469-tiago-pzk',
    'TINI': '1046590-tini',
    'zzoilo': '10171489-zzoilo',
}


In [96]:
shows_world = []

try:
    for spotify_name, songkick_code in DICT_ARTIST_WORLD.items():

        # DEFINO LA URL
        url_request = URL_BASE + songkick_code + GIGOGRAPHY
        # LA OBTENGO
        response = requests.get(url_request)
        html = response.content
        soup = bs(html, "lxml")

        past_concerts = soup.find(id="event-listings").find('ul', class_='event-listings').find_all('li')

        # LISTA DE CONCIERTOS
        for concert in past_concerts:

            show = []

            # LOCALIZO LOS EVENTOS
            try:
                title = concert.get('title')

                if '2022' in title:
                    info_json = concert.find("script", type="application/ld+json")
                    info_json = info_json.get_text(strip=True).replace("\n", "")
                    dict_evento = json.loads(info_json)[0]

                    # FECHA:
                    fecha = dict_evento['startDate']
                  
                    # UBICACION
                    ciudad = dict_evento['location']['address']['addressLocality']
                    pais = dict_evento['location']['address']['addressCountry']

                    show = [spotify_name, fecha, ciudad, pais]

                    shows_world.append(show)

            
            except:
                pass

    # PASO LA BBDD A DATAFRAME
    df = pd.DataFrame(shows_world, columns=['Track_Main_Artist', 'Fecha', 'Ciudad', 'Pais'])
    df.to_csv('concerts_world.csv', sep=';', index=False)
    

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))


In [97]:
shows_spain = []

try:
    for spotify_name, songkick_code in DICT_ARTIST_SPAIN.items():

        # DEFINO LA URL
        url_request = URL_BASE + songkick_code + GIGOGRAPHY
        # LA OBTENGO
        response = requests.get(url_request)
        html = response.content
        soup = bs(html, "lxml")

        past_concerts = soup.find(id="event-listings").find('ul', class_='event-listings').find_all('li')

        # LISTA DE CONCIERTOS
        for concert in past_concerts:

            show = []

            # LOCALIZO LOS EVENTOS
            try:
                title = concert.get('title')

                if '2022' in title:
                    info_json = concert.find("script", type="application/ld+json")
                    info_json = info_json.get_text(strip=True).replace("\n", "")
                    dict_evento = json.loads(info_json)[0]

                    # FECHA:
                    fecha = dict_evento['startDate']

                    # UBICACION
                    ciudad = dict_evento['location']['address']['addressLocality']
                    pais = dict_evento['location']['address']['addressCountry']

                    show = [spotify_name, fecha, ciudad, pais]

                    shows_spain.append(show)

            
            except:
                pass

    # PASO LA BBDD A DATAFRAME
    df = pd.DataFrame(shows_spain, columns=['Track_Main_Artist', 'Fecha', 'Ciudad', 'Pais'])
    df.to_csv('concerts_spain.csv', sep=';', index=False)
    

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))


KeyboardInterrupt: 